In [1]:
from ssm4epi.models.regional_growth_factor import (
    key,
    n_iterations,
    N_mle,
    N_meis,
    N_posterior,
    percentiles_of_interest,
    make_aux,
    dates_full,
    cases_full,
    n_ij,
    n_tot,
    n_pop,
    account_for_nans,
    growth_factor_model,
)

from tqdm.notebook import tqdm
import pandas as pd

import jax.numpy as jnp
import jax
import jax.random as jrn

from isssm.importance_sampling import prediction
from isssm.laplace_approximation import laplace_approximation as LA
from isssm.modified_efficient_importance_sampling import (
    modified_efficient_importance_sampling as MEIS,
)

from pyprojroot.here import here

jax.config.update("jax_enable_x64", True)
from isssm.estimation import initial_theta
import pickle
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
from datetime import date

min_date = date(2021, 4, 3)
max_date = date(2021, 12, 25)

np1 = 10

initial_dates = [
    (d - pd.Timedelta(weeks=np1)).strftime("%Y-%m-%d")
    for d in pd.date_range(start=min_date, end=max_date, freq="W-SAT", inclusive="both")
]
len(initial_dates)

39

In [3]:
GLOBAL_KEY = jrn.PRNGKey(4534365463653)


def f_pred(x, s, y):
    y_county = y[-1]
    y_tot = jnp.minimum(y_county, n_pop).sum()[None]
    growth_factors = s.reshape(-1)
    return jnp.concatenate([y_tot, y_county, growth_factors])


def prediction_pipeline(initial_date):
    (dates_index,) = jnp.where(dates_full == initial_date)[0]
    dates = dates_full[dates_index : dates_index + np1]
    aux = make_aux(initial_date, cases_full, n_ij, n_tot, np1)

    y = aux[0][1:]
    y_nan = y.at[-1].set(jnp.nan)
    missing_inds = jnp.isnan(y_nan)
    theta_manual = jnp.array(
        [5.950e00, -2.063e00, -5.355e00, -4.511e-01, -5.711e-01, 7.932e-01]
    )
    _, y_miss = account_for_nans(
        growth_factor_model(theta_manual, aux), y_nan, missing_inds
    )
    _model_miss = lambda theta, aux: account_for_nans(
        growth_factor_model(theta, aux), y_nan, missing_inds
    )[0]
    theta0_result = initial_theta(
        y_miss,
        _model_miss,
        theta_manual,
        aux,
        n_iter_la=n_iterations,
        options={"maxiter": 10},
    )
    theta0 = theta0_result.x
    fitted_model = _model_miss(theta0, aux)
    proposal_la, _ = LA(y_miss, fitted_model, n_iterations)

    key, subkey = jrn.split(GLOBAL_KEY)
    preds = prediction(
        f_pred,
        y_miss,
        proposal_la,
        fitted_model,
        1000,
        subkey,
        percentiles_of_interest,
        growth_factor_model(theta0, aux),
    )

    result = (theta0, proposal_la, preds, dates, y)

    with open(
        here() / f"data/results/4_local_outbreak_model/results_{initial_date}.pickle",
        "wb",
    ) as f:
        pickle.dump(result, f)

    df = pd.DataFrame(
        {
            "variable": [
                "y_total",
                *[f"y_total_{c}" for c in range(1, 401)],
                *[f"log_rho_{c}_{t}" for t in range(1, 11) for c in range(1, 401)],
            ],
            "c": [0, *range(1, 401), *[c for t in range(1, 11) for c in range(1, 401)]],
            "t": [
                10,
                *jnp.repeat(10, 400),
                *[t for t in range(1, 11) for c in range(1, 401)],
            ],
            "mean": preds[0],
            "sd": preds[1],
            **{
                f"{p * 100:.1f} %": preds[2][i, :]
                for i, p in enumerate(percentiles_of_interest)
            },
        }
    )

    df["date"] = [dates[t - 1] for t in df["t"]]
    df.to_csv(
        here(f"data/results/4_local_outbreak_model/forecasts_{initial_date}.csv"),
        index=False,
    )

In [4]:
def create_forecast_dataframe(initial_date):
    with open(
        here() / f"data/results/4_local_outbreak_model/results_{initial_date}.pickle",
        "rb",
    ) as f:
        result = pickle.load(f)

    (theta0, proposal_la, _, dates, _) = result
    (dates_index,) = jnp.where(dates_full == initial_date)[0]
    dates = dates_full[dates_index : dates_index + np1]
    aux = make_aux(initial_date, cases_full, n_ij, n_tot, np1)

    y = aux[0][1:]
    y_nan = y.at[-1].set(jnp.nan)
    missing_inds = jnp.isnan(y_nan)
    _, y_miss = account_for_nans(growth_factor_model(theta0, aux), y_nan, missing_inds)
    _model_miss = lambda theta, aux: account_for_nans(
        growth_factor_model(theta, aux), y_nan, missing_inds
    )[0]
    fitted_model = _model_miss(theta0, aux)

    key, subkey = jrn.split(GLOBAL_KEY)
    preds = prediction(
        f_pred,
        y_miss,
        proposal_la,
        fitted_model,
        1000,
        subkey,
        percentiles_of_interest,
        growth_factor_model(theta0, aux),
    )
    df = pd.DataFrame(
        {
            "variable": [
                "y_total",
                *[f"y_total_{c}" for c in range(1, 401)],
                *[f"log_rho_{c}_{t}" for t in range(1, 11) for c in range(1, 401)],
            ],
            "c": [0, *range(1, 401), *[c for t in range(1, 11) for c in range(1, 401)]],
            "t": [
                10,
                *jnp.repeat(10, 400),
                *[t for t in range(1, 11) for c in range(1, 401)],
            ],
            "mean": preds[0],
            "sd": preds[1],
            **{
                f"{p * 100:.1f} %": preds[2][i, :]
                for i, p in enumerate(percentiles_of_interest)
            },
        }
    )

    df["date"] = [dates[t - 1] for t in df["t"]]
    df.to_csv(
        here(f"data/results/4_local_outbreak_model/forecasts_{initial_date}.csv"),
        index=False,
    )

In [5]:
import jax.scipy as jsp


def make_theta_df(initial_date):
    with open(
        here() / f"data/results/4_local_outbreak_model/results_{initial_date}.pickle",
        "rb",
    ) as f:
        result = pickle.load(f)

    (theta0, _, _, _, _) = result

    logit_alpha, log_s2_r, log_s2_spat, logit_q, log_Cm1, log_r = theta0

    return pd.Series(
        {
            "alpha": jsp.special.expit(logit_alpha),
            "sigma_r": jnp.sqrt(jnp.exp(log_s2_r)),
            "sigma spatial": jnp.sqrt(jnp.exp(log_s2_spat)),
            "q": jsp.special.expit(logit_q),
            "C": jnp.exp(log_Cm1) + 1,
            "r": jnp.exp(log_r),
            "date": initial_date,
        }
    )

In [ ]:
for initial_date in tqdm(initial_dates):
    prediction_pipeline(initial_date)
    # create_forecast_dataframe(initial_date)

  0%|          | 0/39 [00:00<?, ?it/s]

In [ ]:
pd.DataFrame([make_theta_df(date) for date in initial_dates])

,alpha,sigma_r,sigma spatial,q,C,r,date
0,0.9973384337783732,0.35059824318845784,0.07782708689772433,0.21483234531536205,1.6154071630377946,20.072103640699638,2021-01-23
1,0.9973221054560845,0.35099949171969713,0.0769703414921603,0.22396279959958018,1.6048889091512193,21.280138989337093,2021-01-30
2,0.9973032922658197,0.3508766390253933,0.07794223336215811,0.22784912248207637,1.5862579772657526,21.944387810204827,2021-02-06
3,0.9971516328585379,0.3465211115555963,0.08805430502221483,0.20096494868320028,1.490402031754511,24.335549946944145,2021-02-13
4,0.997190031464567,0.34815179064624396,0.0853877840903443,0.20311575001887627,1.483167773223103,26.399061351963315,2021-02-20
5,0.9972089719431354,0.348935934198242,0.08829881196476037,0.2091723189355267,1.517143283934384,29.39897118794478,2021-02-27
6,0.9972462599107308,0.3491726655207461,0.09022564316473773,0.1973865075141459,1.5434133691864083,31.83313872786039,2021-03-06
7,0.9973217302939098,0.35196492750678005,0.08111408213489478,0.21017419394988054,1.582991447968756,36.66258843502224,2021-03-13
8,0.9973414025170767,0.3518115001191067,0.08079087458752482,0.21953836618546446,1.5649904663898428,36.65706307852924,2021-03-20
9,0.9973379521254647,0.3495975294095747,0.08476406482961753,0.19909087596191624,1.5053706731713234,34.322769938742475,2021-03-27
